Скрипт по преобразованию данных.  Из входных данных в xlsx файле, в котором данные следуют не на границу месяца, с перерывами и переменным периодом, используя линейную интерполяцию по двум соседним точкам создает файл, в которм данные слейдуют с шагом в 1 месяц на первое число, каждого месяца. 
Входные данные должны  быть на листе с именем "Лист1",  

в переменную date - записать название столбца с датой
в переменную value - название столбца с данными 

Реззультат будет записан в файл 
output_file_path = 'interpolated_data.xls'

In [2]:
import pandas as pd
import numpy as np
from datetime import timedelta
from openpyxl import load_workbook

# Чтение данных из файла
file_path = 'ВВП.xlsx'
date='Дата'
value='ВВП'
df = pd.read_excel(file_path, sheet_name='Лист1', parse_dates=[date])

# Создание списка с датами с шагом в месяц
start_date = df[date].min().replace(day=1)
end_date = df[date].max().replace(day=1)
month_range = pd.date_range(start_date, end_date, freq='MS')

# Линейная интерполяция данных
interpolated_data = []
for target_date in month_range:
    # Найти ближайшие точки слева и справа от целевой даты
    left_point = df[df[date] <= target_date].iloc[-1]
    right_point = df[df[date] >= target_date].iloc[0]

    if left_point[date] == right_point[date]:
        interpolated_value = left_point[value]
    else:
        # Рассчитать веса для интерполяции
        left_weight = (right_point[date] - target_date) / (right_point[date] - left_point[date])
        right_weight = 1 - left_weight

        # Рассчитать интерполированное значение
        interpolated_value = left_weight * left_point[value] + right_weight * right_point[value]

    interpolated_data.append([target_date, interpolated_value])

# Создание нового DataFrame
df_interpolated = pd.DataFrame(interpolated_data, columns=[date, value])

# Запись нового листа в файл
output_file_path = 'interpolated_data.xls'
with pd.ExcelWriter(output_file_path, engine='openpyxl', mode='w') as writer:
    df_interpolated.to_excel(writer, sheet_name='Лист1', index=False)

print("Данные успешно интерполированы и сохранены в файле", output_file_path)

Данные успешно интерполированы и сохранены в файле interpolated_data.xls
